# Pre-processing and Modeling
- 1. [Introduction](#1.-Introduction)
- 2. [Pre-processing](#2.-Pre-processing)
- 3. [Modeling](#3.-Modeling)
- 4. [Training](#4.-Training)


In [9]:
import numpy as np

# 1. Introduction

To solve our reinforcement learning problem of navigating a race track, we use a model that learns in an online manner. It aggregates set of experiences (state, action, reward, the subsequent state, and whether or not the episode has terminated) over time and periodically trains its internal model. Therefore, it does not follow a traditional offline learning paradigm and cannot be fully represented in a notebook format. 

However, in this notebook, we can outline our pre-processing and modeling steps and describe how we plan to train our agent. The implementation of these steps can be found in this [repository](https://github.com/uncanny-valley/openai-car-racing-agent).

![Pre-processing diagram](data/diagram.svg)

The approach is as follows:
1. Reset the Gym environment and receive the current state of this environment
2. Pre-process the state into a data structure that can be ingested by our model
3. Repeat and accumulate a buffer of experiences, which stores the current state, the action taken, the reward given for taking this action in the current state, the resulting state, and whether or not the episode has been terminated.
4. Maintain a convolutional neural network to predict the value of taking each action from a given state. Sample a batch from this buffer to train this network. Maintain a duplicate target network to evaluate the predictions and adjust the weights of the first network based on this loss. 
5. Predict each action at each step based on the maximal value given by the network.

# 2. Pre-processing

For our CarRacing-v0 environment, we received a state at some timestep in the form of a frame of size (96, 96, 3). The frame contained 96 by 96 pixels across 3 RGB channels. In our preprocessing [stage](https://github.com/uncanny-valley/openai-car-racing-agent/blob/master/preprocessing.py), we gray-scaled the frame, reducing the RGB channels to one dimension, normalized its pixel values from 0 to 1, and created a queue of consecutive frames of size $\phi$. $\phi$ is configurable on input and attempts to capture the motion of the vehicle. Often in games with high FPS, consecutive frames appear virtually identical. To address this, we dropped frames in between each sub-frame within the queue. Said differently, each sub-frame in the queue is exactly $n$ timesteps away from each other. As a result, we ensure that some progress or motion is being made and properly captured by the queue.

# 3. Modeling

Modeling is accomplished through maintenance of a deep Q-network (DQN). This DQN comrpises of a convolutional neural network (CNN) that takes a pre-processed state (queue of sub-frames) as input and estimates the long-term reward of each action if taken at that state. The action with the maximal value predicted by the CNN will be the action taken by the agent on the next timestep. We follow a decaying $\epsilon$-greedy policy, where the agent has a probability of $\epsilon$ of taking a random action, which decays over time i.e. the agent becomes more certain as time passes.

#  4. Training

Training is organized by epochs and episodes. On start, we can define the number of epochs we'd like to train the agent for and how many timesteps each epoch should have. The agent then simulates several episodes of attempting to navigate the race track. Each episode contains timesteps and eventually terminates once the vehicle completes the track. Once the sum of each episode's timesteps surpasses the defined number of timesteps per epoch, the epoch terminates and a new epoch begins. 

At each timestep, the agent makes a forward and backward pass through its CNN. The loss is calculated by taking the mean squared error of the predicted action values from the CNN and the predicted action values from a duplicate CNN called the target network. The target network is identical to the main CNN but it takes the next state as input and tries to predict its best action-value. We use a second network to add stability to the predictions of the next state values. The weights of the target network are updated periodically throughout the training process.



# Results

Models are evaluated by the mean average total reward over 100 episodes. An environment is "solved" if it receives over 900 reward out of a possible 1000. Our model doesn't perform up to this standard but it still yields a solid performance:

In [10]:
with open('../metrics/agent-2021-09-06-23-06-result-2021-09-08-04-33.txt', 'r') as f:
    values = []
    for line in f.readlines():
        values.append(float(line.strip()))
    
    print(f'Mean average total reward: {np.mean(values)}')

Mean average total reward: 832.8247046812999
